<a href="https://colab.research.google.com/github/numagic/lumos/blob/dev/tutorials/colab/Laptime_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Usually it is recommended to use lumos with the docker image it provides. But we can also run `lumos` with conda environment.

Google Colab provides free GPU and TPU VMs that one could use with jupyter notebook style UI, and this is what we're going to use.

To set up the environment, we'll:
1) install conda on google colab using `condacolab`
2) clone the `lumos` git repo, and setup the conda environment (this will be replaced by pip install in the future)
3) run laptime simulation example

# Install Conda on Google Colab

<!-- By Jaime Rodríguez-Guerra <@jaimergp>. Last modified 2021.08.04 -->

`condacolab` simplifies the setup as much as possible, but there are some gotchas.

**⚠️ Read this before continuing!**

* The `condacolab` commands need to be run the first Code cell!
* Once you run `condacolab.install()`, the Python kernel will be restarted. This is **normal and expected**. After that, you can continue running the cells below like normal.
* Do not use the `Run all` option. Run the `condacolab` cell _individually_ and wait for the kernel to restart. **Only then**, you can run all cells if you want.
* You can only use the `base` environment. Do not try to create new ones; instead update `base` with either:
  * `conda install <packages>`
  * `conda env update -n base -f environment.yml`
* If you want to use GPUs, make sure you are using such an instance before starting!
* If you get an error, please raise an issue [here](https://github.com/jaimergp/condacolab/issues).

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:27
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Make sure cuda is available
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Thu Mar 24 21:01:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |            

# Setup `lumos` environment

In [3]:
# Clone repo and enter the right path
!git clone https://github.com/numagic/lumos.git
%cd lumos
!git checkout dev
!ls

Cloning into 'lumos'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 383 (delta 141), reused 297 (delta 72), pack-reused 0
Receiving objects: 100% (383/383), 671.99 KiB | 5.84 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/lumos
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
data			environment.yml  pyproject.toml  tutorials
docker			examples	 README.md
docker-compose.gpu.yml	imgs		 setup.py
docker-compose.yml	lumos		 tests


In [4]:
# Colab by default uses python 3.7, which we can't change. Condacolab also only
# supports the base env, which we update.
# It may take 4-5 minutes to set up the conda environment, particularly
# with setting up the cuda toolkit for the GPU.
# !conda env update -n base -f environment.yml

# Or... we could direclty install them which seems faster than asking conda to
# solve for the environment. So we'll take this to make the colab experience
# better.
# TODO: make dependency automatic -> this would require conda as there are
# non-python dependencies
!conda install -c conda-forge cyipopt
!pip install casadi pyarrow pandas
# Install the GPU version of jax to use GPU (need correct cuda version)
!pip install jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_releases.html
# install lumos
!pip install -i https://test.pypi.org/simple/ numagic-lumos==0.0.5a0

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1006         1.1 MB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    conda-4.12.0               |   py37h89c1867_0         1.0 MB  conda-forge
    cyipopt-1.1.0              |   py37h4f1d67c_1         144 KB  conda-forge
    future-0.18.2              |   py37h89c1867_4         713 KB  conda-forge
    ipopt-3.14.5               |    

Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 102 kB 3.8 MB/s 
Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     |████████████████████████████████| 155.4 MB 19 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.2
    Uninstalling jaxlib-0.3.2:
      Successfully uninstalled jaxlib-0.3.2
Looking in indexes: https://test.pypi.org/simple/


In [5]:
# test jax on GPU, if you see a spike in the GPU ram used -> yes, you're using GPU
# If you see warnings about GPU not found, then either the VM connected has no
# GPU or the support packages are not installed correctly
import jax
import jax.numpy as jnp
import numpy as np
a = np.random.randn(100, 100)
b = np.random.randn(100, 100)
c = jnp.dot(a, b)
print(jax.devices())
del a, b, c

[GpuDevice(id=0, process_index=0)]


# Run laptime simulation example

Note that unfortunately colab does not show the stdout printed to the terminal, therefore the user must use the command tabs: 'Runtime' -> 'View runtime logs' to see the stdout outputs, such as those from IPOPT.

In [6]:
import logging
import sys

use_gpu_with_jax = True
is_cyclic = True
backend = "jax" # supports jax and casadi

def main():
    import jax
    import os
    import cyipopt
    cyipopt.set_logging_level(logging.WARN)

    from lumos.models.composition import ModelMaker
    from lumos.models.simple_vehicle_on_track import SimpleVehicleOnTrack
    from lumos.models.tires.utils import create_params_from_tir_file
    from lumos.simulations.laptime_simulation import LaptimeSimulation

    TRACK_DIR = "data/tracks"

    if is_cyclic:
        initial_states = {"time": 0}
    else:
        initial_states = {"time": 0, "n": 0, "eta": 0, "vx": 50, "vy": 0, "yaw_rate": 0}
    
    # Usually GPUs are designed to operate with float32 or even float16, and are
    # much slower with doubles (float64). Here we stick with float64 to ensure
    # we have the same results as 64bit as with casadi backend.
    if use_gpu_with_jax:
        jax.config.update('jax_platform_name', 'gpu')
        os.environ['JAX_PLATFORM_NAME'] = 'GPU'
        jax.config.update("jax_enable_x64", True)
    else:
        # somehow jax doesn't see the cpu device on colab?!
        jax.config.update('jax_platform_name', 'cpu')
        os.environ['JAX_PLATFORM_NAME'] = 'CPU'
        jax.config.update("jax_enable_x64", True)


    track = "Catalunya"
    track_file = os.path.join(TRACK_DIR, track + ".csv")

    model_config = SimpleVehicleOnTrack.get_recursive_default_model_config()

    # EXAMPLE: change tire model
    # model_config.replace_subtree("vehicle.tire", ModelMaker.make_config("PerantoniTire"))

    # EXMAPLE: change an aero model
    # model_config.replace_subtree("vehicle.aero", ModelMaker.make_config("MLPAero"))

    model = SimpleVehicleOnTrack(model_config=model_config)
    params = model.get_recursive_default_params()

    # Example of changing model parameters
    # TODO: an issue here is that we need to instantiate the model first to get params
    # but that's unavoidable because without the model, we don't even know the tree
    # structure of all the submodels, let alone the default parameters.
    # params.set_param("vehicle.vehicle_mass", 1700)

    # Example: change tire parameters
    sharpened_params = create_params_from_tir_file("data/tires/sharpened.tir")
    # FIXME: here we're using private methods. We should probably add a method to change
    # the parameters of an entire node in the ParameterTree
    tire_params = params._get_subtree("vehicle.tire")
    tire_params._data = sharpened_params
    params.replace_subtree("vehicle.tire", tire_params)

    final_outputs = {}
    final_states = {}
    ocp = LaptimeSimulation(
        model_params=params,
        model_config=model_config,
        sim_config=LaptimeSimulation.get_sim_config(
            num_intervals=250,
            hessian_approximation="exact",
            is_cyclic=is_cyclic,
            is_condensed=False,
            backend=backend,
            track=track_file,
            transcription="LGR",
            initial_states=initial_states,
            final_outputs=final_outputs,
            final_states=final_states,
            logging_config={
                "results_dir": "results",  # store in a new directory at current directory
                "sim_name": track,
                "log_final_iter": False,
                "log_metrics_history": False,
                "log_every_nth_iter": 0,  # if 0, logging is off
            },
        ),
    )

    x0 = ocp.get_init_guess()


    print("starting the first solve!")
    solution, info = ocp.solve(
        x0,
        max_iter=200,
        print_level=5,
        print_timing_statistics="yes",
        print_info_string="yes",
        derivative_test="none",
        dual_inf_tol=1e-3,
        constr_viol_tol=1e-3,
    )
    total_time = ocp.dec_var_operator.get_var(
        solution, group="states", name="time", stage=-1
    )
    print(info["status_msg"])
    print(f"finished in {info['num_iter']} iterations")
    print(f"Maneuver time {total_time:.3f} sec")

    # # We can change the parameters and solve again
    # ocp.modify_model_param("vehicle.vehicle_mass", 2100.0)
    # print("starting the second solve!")
    # solution, info = ocp.solve(
    #     solution,
    #     max_iter=200,
    #     print_level=5,
    #     print_timing_statistics="yes",
    #     print_info_string="yes",
    #     derivative_test="none",
    #     dual_inf_tol=1e-3,
    #     constr_viol_tol=1e-3,
    # )
    # total_time = ocp.dec_var_operator.get_var(
    #     solution, group="states", name="time", stage=-1
    # )
    # print(f"Maneuver time {total_time:.3f} sec")

# timing (note: this could be rather unstable due to the VM and resources available)
# with 1000 intervals:
# casadi:           89 iter 89.3sec in NLP, 71.7 in IPOPT
# JAX GPU (64bit):  89 iter 12.6sec in NLP functions, 87.4sec in IPOPT
# MUMPS -> MA57 will dramatically reduce the IPOPT overhead
# condensing could also reduce IPOPT overhead
# JAX potentially uses less sparsity info than casadi functions -> more IPOPT time
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
main()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
DEBUG:lumos.models.tires.utils:FILE_TYPE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FILE_FORMAT is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:LENGTH is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FORCE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:ANGLE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:MASS is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:TIME is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:TYRESIDE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FILE_TYPE is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FILE_FORMAT is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:LENGTH is not a numeric value and is discarded.
DEBUG:lumos.models.tires.utils:FORCE is not a numeric value and is 

starting the first solve!


INFO:lumos.optimal_control.nlp:model_algebra.constraints: 2.852677
INFO:lumos.optimal_control.nlp:model_algebra.jacobian: 10.648831
INFO:lumos.optimal_control.nlp:model_algebra.hessian: 23.436360
INFO:lumos.optimal_control.nlp:continuity.constraints: 0.005650
INFO:lumos.optimal_control.nlp:continuity.jacobian: 0.000004
INFO:lumos.optimal_control.nlp:continuity.hessian: 0.000009
INFO:lumos.optimal_control.nlp:cyclic.constraints: 0.000258
INFO:lumos.optimal_control.nlp:cyclic.jacobian: 0.000004
INFO:lumos.optimal_control.nlp:cyclic.hessian: 0.000006
INFO:lumos.optimal_control.scaled_mesh_ocp:Triggering jax JIT completed
INFO:lumos.optimal_control.scaled_mesh_ocp:Time NLP execution
INFO:lumos.optimal_control.nlp:time.objective: 0.000012
INFO:lumos.optimal_control.nlp:time.gradient: 0.000205
INFO:lumos.optimal_control.nlp:time.hessian: 0.000003
INFO:lumos.optimal_control.nlp:inputs_penalty.objective: 0.000829
INFO:lumos.optimal_control.nlp:inputs_penalty.gradient: 0.001516
INFO:lumos.optim

b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'
finished in 64 iterations
Maneuver time 115.372 sec


## Visualiza results
TODO: add some nice plots/animations